In [81]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [82]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [99]:
# Hyper-parameters
num_epochs = 5
batch_size = 16
learning_rate = 0.0001

In [100]:
# dataset has PILImage images of range [0, 1]
# Transform them to tensors of normalised range [-1, 1]

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # Subtract mean (first tuple) then divide by SD (second tuple)

In [101]:
train_dataset = torchvision.datasets.CIFAR10(root='./data/CIFAR10', 
                                             train=True, 
                                             download=True, 
                                             transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data/CIFAR10', 
                                             train=False, 
                                             download=True, 
                                             transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [102]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [103]:
# Conv Net
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # Fully connected layers
        self.fc1 = nn.Linear(16*5*5, 120) # 5x5 is image size before fully connected layer
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

In [104]:
model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [105]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # original shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        
        if (i+1) % 750 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print("Finished Training")

Epoch [1/5], Step [750/3125], Loss: 2.1018
Epoch [1/5], Step [1500/3125], Loss: 1.5607
Epoch [1/5], Step [2250/3125], Loss: 1.9203
Epoch [1/5], Step [3000/3125], Loss: 1.4957
Epoch [2/5], Step [750/3125], Loss: 2.0396
Epoch [2/5], Step [1500/3125], Loss: 1.6291
Epoch [2/5], Step [2250/3125], Loss: 1.6693
Epoch [2/5], Step [3000/3125], Loss: 1.8794
Epoch [3/5], Step [750/3125], Loss: 1.2578
Epoch [3/5], Step [1500/3125], Loss: 1.4567
Epoch [3/5], Step [2250/3125], Loss: 1.2840
Epoch [3/5], Step [3000/3125], Loss: 1.3012
Epoch [4/5], Step [750/3125], Loss: 1.7850
Epoch [4/5], Step [1500/3125], Loss: 1.5894
Epoch [4/5], Step [2250/3125], Loss: 1.3376
Epoch [4/5], Step [3000/3125], Loss: 1.5599
Epoch [5/5], Step [750/3125], Loss: 1.3078
Epoch [5/5], Step [1500/3125], Loss: 1.2329
Epoch [5/5], Step [2250/3125], Loss: 1.4069
Epoch [5/5], Step [3000/3125], Loss: 1.4277
Finished Training


In [106]:
# Testing
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        _, preds = torch.max(outputs, axis=1)
        n_samples += labels.size(dim=0)
        n_correct += (preds == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = preds[i]
            if label == pred:
                n_class_correct[label] += 1
            n_class_samples[label] += 1
    
    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc}%')
    
    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc}%')

Accuracy of the network: 50.3%
Accuracy of plane: 57.7%
Accuracy of car: 65.7%
Accuracy of bird: 49.7%
Accuracy of cat: 20.6%
Accuracy of deer: 28.4%
Accuracy of dog: 38.8%
Accuracy of frog: 68.5%
Accuracy of horse: 66.0%
Accuracy of ship: 53.7%
Accuracy of truck: 53.9%
